In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math as mt
import random as rng

from sklearn import preprocessing

Perform the initial loading of the data. Cycle column is used as the labels, otherwise the simulation number and input variable columns are dropped. Also load the test dataset available in case we want to use it

In [2]:

#Get a range of the column values so they can be used as the column names
def getColumns(size):
    columns = []
    for i in range(size):
        columns.append(i)
    return columns

def initialLoad(directory):
    #Read the csv file, discard the null columns
    turbo = pd.read_csv(directory, header=None, sep=' ')
    turbo = turbo.drop([26, 27], axis=1)
    #Drop the simulation and cycle count from the data
    turboSens = turbo.drop([0,1,2,3,4], axis=1)
    turboSens.columns = getColumns(turboSens.shape[1])
    return turbo, turboSens


#Read the csv file, discard the null columns
directory = "../CMAPSSData/train_FD001.txt"
directoryTest = "../CMAPSSData/test_FD001.txt"
directoryTestExtra = "../CMAPSSData/RUL_FD001.txt"

turbo,turboSens = initialLoad(directory)

valTurbo,valSens = initialLoad(directoryTest)


In [3]:
#minVals = []
#maxVals = []
# Get the minimum and maximum value of each column and store them in minVals and maxVals respectively
#for i in range(len(turboSens.columns)):
#    minVals.append(turboSens[i].min())
#    maxVals.append(turboSens[i].max())
    

Gather the cycle numbers for the dataset and convert them into labels for the model

In [4]:
turboUsefulSens = turboSens
valUsefulSens = valSens

turboNp = turboUsefulSens.to_numpy()
valNp = valUsefulSens.to_numpy()


In [5]:
#Get the number of cycles for each simulation, and split each simulation
maxCycles = []
simCount = 100

def getTargets(cycleList, count):
    simulations = []
    current = 0
    for i in range(simCount):
        sim = [cycleList[current]]
        current += 1
        while current < len(cycleList):
            if(cycleList[current]==1):
                break
            sim.append(cycleList[current])
            current+=1
        simulations.append(sim)
    return simulations

simulations = getTargets(turbo[1], simCount)


In [6]:
#Get the lowest cycle max
maxCycles = []
for i in simulations:
    maxCycles.append(max(i))
    
print(min(maxCycles))

128


In [7]:
#Flip the cycle counts so they represent deterioration instead of incrementing(decrementing from total cycles to 0 instead of incrementing from 0 to total cycles)
simulationsRev = []
for sim in simulations:
    simRev = []
    maxSim = max(sim)
    for i in sim:
        simRev.append(maxSim - i)
    simulationsRev.append(simRev)

In [8]:
flat_list = []
for sim in simulationsRev:
    for item in sim:
        flat_list.append(item)
        
flatNp = np.array(flat_list)

Save the data for the next stage

In [9]:
np.save("Train", turboNp)
np.save("TrainTarget", flatNp)

Do the same process for the test data in case we want to use it. takes a few extra steps since the labels need another file to mark the RUL when it stops 

In [10]:
valTargets = getTargets(valTurbo[1],simCount)
file = open(directoryTestExtra, 'r')

valTargetRev = []
for sim in valTargets:
    rev = []
    maxCount = max(sim)
    extra = int(file.readline())
    for i in sim:
        rev.append(maxCount - i + extra)
    valTargetRev.append(rev)
file.close()

flat_val = []
for sim in valTargetRev:
    for item in sim:
        flat_val.append(item)
        
        
valTargNp = np.array(flat_val)

In [11]:
np.save("Validate", valNp)
np.save("ValidateTarget", valTargNp)

Also put all the data in the same file in case we want to use all the data combined

In [12]:
allSensNp = np.append(turboNp, valNp, axis=0)
allTargNp = np.append(flatNp, valTargNp)

print(len(allSensNp))
print(len(allTargNp))

33727
33727


In [13]:
np.save("all", allSensNp)
np.save("allTarget", allTargNp)